In [33]:
qkp100_str = ''
qkp200_str = ''
qkp300_str = ''
qkp400_str = ''
qkp500_str = ''

with open('../qkpinstances/100_R_G_DR_25.txt') as f:
    qkp100_str = f.read()

with open('../qkpinstances/200_R_G_DR_25.txt') as f:
    qkp200_str = f.read()

with open('../qkpinstances/300_R_G_DR_25.txt') as f:
    qkp300_str = f.read()

with open('../qkpinstances/400_R_G_DR_25.txt') as f:
    qkp400_str = f.read()

with open('../qkpinstances/500_R_G_DR_25.txt') as f:
    qkp500_str = f.read()

In [18]:
import numpy as np

def parse_qkp(qkp_str):
    parts = qkp_str.split('\n')
    items = int(parts[0])
    values = np.zeros((items, items), dtype=int)
    lin_values = parts[1].split(' ')
    for i in range(0, items):
        values[i][i] = int(lin_values[i])
    quad_values = [row.split(' ') for row in parts[2:(2 + items - 1)]]
    for i in range(0, items - 1):
        for j in range(0, items - i - 1):
            values[i][i + j + 1] = int(quad_values[i][j])
            values[i + j + 1][i] = int(quad_values[i][j])
    capacity = int(parts[2 + items + 1])
    weights = np.zeros(items, dtype=int)
    for i, w in enumerate(parts[2 + items + 2].split(' ')[1:]):
        weights[i] = w
    return { 'items': items, 'values': values, 'weights': weights, 'capacity': capacity }

In [34]:
qkp100 = parse_qkp(qkp100_str)
qkp200 = parse_qkp(qkp200_str)
qkp300 = parse_qkp(qkp300_str)
qkp400 = parse_qkp(qkp400_str)
qkp500 = parse_qkp(qkp500_str)

In [35]:
from qiskit_optimization import QuadraticProgram

def qkp_to_qp(qkp):
    model = QuadraticProgram('QKP')
    variables = ['x_' + str(i) for i in range(0, qkp['items'])]
    for var in variables:
        model.binary_var(name=var)
    linear_cons = {}
    for i in range(0, qkp['items']):
        linear_cons['x_' + str(i)] = qkp['weights'][i]
    model.linear_constraint(linear=linear_cons, sense='<=', rhs=qkp['capacity'], name='weight_constraint')
    linear_obj = {}
    for i in range(0, qkp['items']):
        linear_obj['x_' + str(i)] = qkp['values'][i][i]
    quad_obj = {}
    for i in range(0, qkp['items'] - 1):
        for j in range(i + 1, qkp['items']):
            quad_obj[('x_' + str(i), 'x_' + str(j))] = qkp['values'][i][j]
    model.maximize(constant=0, linear=linear_obj, quadratic=quad_obj)
    return model

In [36]:
qp100_model = qkp_to_qp(qkp100)
qp200_model = qkp_to_qp(qkp200)
qp300_model = qkp_to_qp(qkp300)
qp400_model = qkp_to_qp(qkp400)
qp500_model = qkp_to_qp(qkp500)

In [39]:
from qiskit_optimization.converters import InequalityToEquality
from qiskit_optimization.converters import IntegerToBinary
from qiskit_optimization.converters import LinearEqualityToPenalty

def qp_to_qubo(qp_model):
    ineq2eq = InequalityToEquality()
    int2bin = IntegerToBinary()
    lineq2penalty = LinearEqualityToPenalty()
    qp_eq = ineq2eq.convert(qp_model)
    qp_eq_bin = int2bin.convert(qp_eq)
    qubo = lineq2penalty.convert(qp_eq_bin)
    return qubo

qubo100 = qp_to_qubo(qp100_model)
qubo200 = qp_to_qubo(qp200_model)
qubo300 = qp_to_qubo(qp300_model)
qubo400 = qp_to_qubo(qp400_model)
qubo500 = qp_to_qubo(qp500_model)

In [40]:
import math

def count_quads(qkp):
    slacks = math.ceil(math.log2(qkp['capacity']))
    total = slacks + qkp['items']
    return int(total * (total - 1) / 2)

print(count_quads(qkp100), count_quads(qkp200), count_quads(qkp300), count_quads(qkp400), count_quads(qkp500))

6216 23871 49455 86736 134940


In [41]:
from qiskit_algorithms.utils import algorithm_globals
from qiskit_algorithms import QAOA, NumPyMinimumEigensolver
from qiskit_algorithms.optimizers import COBYLA
from qiskit.primitives import Sampler
from qiskit_optimization.algorithms import (
    MinimumEigenOptimizer,
    RecursiveMinimumEigenOptimizer,
    SolutionSample,
    OptimizationResultStatus,
)
from qiskit.visualization import plot_histogram
from qiskit.circuit.library.n_local.qaoa_ansatz import QAOAAnsatz

algorithm_globals.random_seed = 10598
qaoa_mes = QAOA(sampler=Sampler(), optimizer=COBYLA())
exact_mes = NumPyMinimumEigensolver()
qaoa = MinimumEigenOptimizer(qaoa_mes)
exact = MinimumEigenOptimizer(exact_mes)
#op, offset = qubo.to_ising()
#exact_result = exact.solve(qubo)
#qaoa_result = qaoa.solve(qubo)
#print(qaoa_result.prettyprint())
#qaoa_circuit = QAOAAnsatz(op, 1)
#transpiled_circuit = transpile(qaoa_circuit, backend)
#fake_transpiled_circuit = transpile(qaoa_circuit, fake_backend)
#print(transpiled_circuit.depth(), fake_transpiled_circuit.depth())
#print(transpiled_circuit.count_ops(), fake_transpiled_circuit.count_ops())

In [47]:
from qiskit.providers.fake_provider import FakeWashingtonV2, FakeSherbrooke, FakeCambridgeV2
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit import QuantumCircuit, transpile

service = QiskitRuntimeService()
fake_backend = FakeWashingtonV2()
backend = service.backend('ibm_kyoto')

In [66]:
def plot_qkp100(qkp, backend, fake_backend):
    for i in range(50, 101, 10):
        qkp['items'] = i
        qp = qkp_to_qp(qkp)
        qubo = qp_to_qubo(qp)
        op, offset = qubo.to_ising()
        quads = count_quads(qkp)
        qaoa_circuit = QAOAAnsatz(op, 1)
        transpiled_circuit = transpile(qaoa_circuit, backend)
        depth = transpiled_circuit.depth()
        gate_counts = transpiled_circuit.count_ops()
        average_swaps = (gate_counts['ecr'] - quads * 2) / 3
        fake_transpiled_circuit = transpile(qaoa_circuit, fake_backend)
        fake_depth = fake_transpiled_circuit.depth()
        fake_gate_counts = fake_transpiled_circuit.count_ops()
        fake_average_swaps = (fake_gate_counts['cx'] - quads * 2) / 3
        print(f"Items: {i}, Depth_ECR: {depth}, ECR_Gates: {gate_counts['ecr']}, Average_Swap_Overhead: {average_swaps}")
        print(f"Items: {i}, Depth_CNOT: {fake_depth}, CNOT_Gates: {fake_gate_counts['cx']}, Average_Swap_Overhead: {fake_average_swaps}")

qkp100 = parse_qkp(qkp100_str)
plot_qkp100(qkp100, backend, fake_backend)
qkp100 = parse_qkp(qkp100_str)

Items: 50, Depth_ECR: 8532, ECR_Gates: 11834, Average_Swap_Overhead: 2684.0
Items: 50, Depth_CNOT: 2783, CNOT_Gates: 11131, Average_Swap_Overhead: 2449.6666666666665
Items: 60, Depth_ECR: 12819, ECR_Gates: 16692, Average_Swap_Overhead: 3860.0
Items: 60, Depth_CNOT: 3184, CNOT_Gates: 14467, Average_Swap_Overhead: 3118.3333333333335
Items: 70, Depth_ECR: 14736, ECR_Gates: 21699, Average_Swap_Overhead: 5019.0
Items: 70, Depth_CNOT: 3921, CNOT_Gates: 19946, Average_Swap_Overhead: 4434.666666666667
Items: 80, Depth_ECR: 20352, ECR_Gates: 28250, Average_Swap_Overhead: 6626.0
Items: 80, Depth_CNOT: 5767, CNOT_Gates: 24795, Average_Swap_Overhead: 5474.333333333333
Items: 90, Depth_ECR: 18703, ECR_Gates: 33096, Average_Swap_Overhead: 7598.0
Items: 90, Depth_CNOT: 5890, CNOT_Gates: 32633, Average_Swap_Overhead: 7443.666666666667
Items: 100, Depth_ECR: 25023, ECR_Gates: 39981, Average_Swap_Overhead: 9183.0
Items: 100, Depth_CNOT: 7560, CNOT_Gates: 38575, Average_Swap_Overhead: 8714.333333333334


In [65]:
def plot_qkp100_slack_optimized(qkp, backend, fake_backend):
    for i in range(0, 10):
        qp = qkp_to_qp(qkp)
        qubo = qp_to_qubo(qp)
        op, offset = qubo.to_ising()
        quads = count_quads(qkp)
        qaoa_circuit = QAOAAnsatz(op, 1)
        transpiled_circuit = transpile(qaoa_circuit, backend)
        depth = transpiled_circuit.depth()
        gate_counts = transpiled_circuit.count_ops()
        average_swaps = (gate_counts['ecr'] - quads * 2) / 3
        fake_transpiled_circuit = transpile(qaoa_circuit, fake_backend)
        fake_depth = fake_transpiled_circuit.depth()
        fake_gate_counts = fake_transpiled_circuit.count_ops()
        fake_average_swaps = (fake_gate_counts['cx'] - quads * 2) / 3
        print(f"Items: 100, Depth_ECR: {depth}, ECR_Gates: {gate_counts['ecr']}, Average_Swap_Overhead: {average_swaps}")
        print(f"Items: 100, Depth_CNOT: {fake_depth}, CNOT_Gates: {fake_gate_counts['cx']}, Average_Swap_Overhead: {fake_average_swaps}")
        qkp['capacity'] = int(qkp['capacity'] / 2)
        qkp['weights'] = (qkp['weights'] / 2).astype(int)

qkp100 = parse_qkp(qkp100_str)
plot_qkp100_slack_optimized(qkp100, backend, fake_backend)
qkp100 = parse_qkp(qkp100_str)

Items: 100, Depth_ECR: 38265, ECR_Gates: 42102, Average_Swap_Overhead: 9890.0
Items: 100, Depth_CNOT: 7348, CNOT_Gates: 39356, Average_Swap_Overhead: 8974.666666666666
Items: 100, Depth_ECR: 23961, ECR_Gates: 40344, Average_Swap_Overhead: 9378.0
Items: 100, Depth_CNOT: 9693, CNOT_Gates: 40969, Average_Swap_Overhead: 9586.333333333334
Items: 100, Depth_ECR: 14920, ECR_Gates: 34952, Average_Swap_Overhead: 7654.0
Items: 100, Depth_CNOT: 7525, CNOT_Gates: 38717, Average_Swap_Overhead: 8909.0
Items: 100, Depth_ECR: 17519, ECR_Gates: 36657, Average_Swap_Overhead: 8295.0
Items: 100, Depth_CNOT: 6010, CNOT_Gates: 34942, Average_Swap_Overhead: 7723.333333333333
Items: 100, Depth_ECR: 20193, ECR_Gates: 38310, Average_Swap_Overhead: 8918.0
Items: 100, Depth_CNOT: 7240, CNOT_Gates: 36819, Average_Swap_Overhead: 8421.0
Items: 100, Depth_ECR: 21110, ECR_Gates: 34955, Average_Swap_Overhead: 7871.0
Items: 100, Depth_CNOT: 7464, CNOT_Gates: 35882, Average_Swap_Overhead: 8180.0
Items: 100, Depth_ECR: 25

In [48]:
circuit = QuantumCircuit(2)
circuit.rzz(qubit1=0, qubit2=1, theta=math.pi * 0.25)

print(transpile(circuit, fake_backend))

                                        
          q_0 -> 0 ──■───────────────■──
                   ┌─┴─┐┌─────────┐┌─┴─┐
          q_1 -> 1 ┤ X ├┤ Rz(π/4) ├┤ X ├
                   └───┘└─────────┘└───┘
    ancilla_0 -> 2 ─────────────────────
                                        
    ancilla_1 -> 3 ─────────────────────
                                        
    ancilla_2 -> 4 ─────────────────────
                                        
    ancilla_3 -> 5 ─────────────────────
                                        
    ancilla_4 -> 6 ─────────────────────
                                        
    ancilla_5 -> 7 ─────────────────────
                                        
    ancilla_6 -> 8 ─────────────────────
                                        
    ancilla_7 -> 9 ─────────────────────
                                        
   ancilla_8 -> 10 ─────────────────────
                                        
   ancilla_9 -> 11 ─────────────────────
                

In [213]:
circuit = QuantumCircuit(2)
circuit.swap(qubit1=0, qubit2=1)
print(transpile(circuit, backend).depth())

14


In [222]:
FakeCambridgeV2().operation_names

['id', 'u1', 'u2', 'u3', 'cx', 'measure', 'delay']